In [77]:
#not needed ... so sad i cant reuse this masterpiece... ;) 
def table_grid(amount):
    #x=1
    y=0
    row_length = 4
    for i in range(amount):
        mod = i % row_length
        if(mod==0):
            y+=1
        x = mod +1
        print(x,y)

In [221]:
import numpy as np
import scipy.io
#from scipy import io
import matplotlib.pyplot as plt
%matplotlib inline
import random
import math
import io
from scipy import optimize, special
import sklearn.preprocessing
import sklearn.linear_model 

In [8]:
vars = {}
a = scipy.io.loadmat("ex3data1.mat",vars)
X = vars['X']
y = vars['y']
np.place(y,y==10,0)
m = len(X)
theta = np.zeros((X.shape[1]+1,1))
X = np.concatenate((np.ones((m,1)),X),axis=1)

In [6]:
theta.shape

(401, 1)

Plot the data

In [7]:
tst = X[100].reshape((20,20))
plt.imshow(tst,cmap='Greys_r')

ValueError: total size of new array must be unchanged

In [10]:
from IPython.display import Image
from IPython.display import display
import PIL.Image

def showarray(a, fmt='jpeg'):
    a = a[:,np.newaxis]
    a = a.reshape((20,20))
    a = a*255
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue(),width=100,height=100))

def plot_amount(dataset,amount):
    random.seed()
    for x in range(amount):
        i = random.randint(0,dataset.shape[0]-1)
        showarray(X[i])
    
    
    

In [83]:
def plot_random(dataset,amount):
    #grid_width = round(math.sqrt(amount))
    grid_height = 3
    random.seed()
    fig = plt.figure()
    for x in range(0,amount-1):
        i = random.randint(0,dataset.shape[0]-1)
        fig.add_subplot(grid_height,2,x+1)
        img = dataset[i].reshape((20,20))
        imgplot = plt.imshow(img,cmap='Greys_r')

In [84]:
showarray(X[11])

# Next section starting :: Multiclass logistic regression 

next section

In [13]:
theta = np.zeros((X.shape[1],1))


In [119]:
def predict(X,theta):
    return special.expit(np.dot(X,theta))

In [274]:
def regularized_cost(theta,dataset,labels,gamma=0.0001):
    m = len(dataset)
    if len(theta.shape) == 1:
        theta = theta[:,np.newaxis]
    prediction = predict(dataset,theta)
    regularizer = np.sum(np.delete(theta,0,0))
    cost = -1/m*np.sum(np.dot((-labels.T),np.log(prediction))-np.dot((1-labels).T,np.log(1-prediction)))+(gamma/(2*m))*regularizer
    return cost

In [275]:
def regularized_gradient(theta,dataset,labels,gamma=0.0001):
    m = len(dataset)
    if len(theta.shape) == 1:
        theta = theta[:,np.newaxis]
    regularizer = (gamma/m) * (np.insert(np.delete(theta,0,0),0,0,0))
    #print(regularizer.shape)
    grad = (1/m * ((np.dot(dataset.T,(predict(dataset,theta)-labels))))) + regularizer
    flat_grad= grad.flatten()
    return flat_grad


In [276]:
def minimize(theta,dataset,labels):
    #return optimize.fmin(regularized_cost,theta)
    return optimize.fmin_tnc(regularized_cost,theta,fprime=regularized_gradient,args=(dataset,labels))

In [277]:
def train(examples,labels,class_nr):
    #examples = sklearn.preprocessing.normalize(examples)
    new_labels = np.copy(labels)
    np.place(new_labels,new_labels==class_nr,1) 
    np.place(new_labels,new_labels!=1,0)
    theta = np.zeros((examples.shape[1],1))
    return minimize(theta = theta,dataset=examples,labels=new_labels)
    

    

In [278]:
for i in range(0,10):
    if i == 0:
        all_theta = train(X,y,i)[0]
        all_theta = all_theta[:,np.newaxis]
    else:
        curr_theta = train(X,y,i)[0]
        curr_theta = curr_theta[:,np.newaxis]
        all_theta = np.concatenate((all_theta,curr_theta),axis=1)
    all_theta
    

/home/jaidmin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log


In [279]:
all_theta.shape

(401, 10)

In [280]:
X.shape

(5000, 401)

In [281]:
p = predict(X,all_theta)
p.shape

(5000, 10)

In [282]:
y_hat = np.argmax(p,axis=1)
y_hat = y_hat[:,np.newaxis]



In [283]:
y_hat.shape

(5000, 1)

In [284]:
difference = y - y_hat

In [285]:
errors = 0
kind_of_error = []

In [286]:
for i in range(0,len(y)-1):
    if difference[i] != 0:
        errors +=1
        kind_of_error.append((y_hat[i],y[i]))

In [287]:
(len(y)-errors) / len(y)

0.694

In [249]:
errors

400

In [251]:
clf = sklearn.linear_model.LogisticRegression(C=2)

In [252]:
clf.fit(X,y[:,0])

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [253]:
y_hat_true = clf.predict(X)